In [16]:
%reload_ext autoreload
%autoreload 2
%load_ext jupyter_black

from tqdm import tqdm
import sys
from pathlib import Path

sys.path.append("..")

import torch
from src.datautil import *
from src.modelutil import *
from src.trainutil import *
from src.metrics import *

import yaml

data_root = "../data"
device = "cuda" if torch.cuda.is_available() else "cpu"

The jupyter_black extension is already loaded. To reload it, use:
  %reload_ext jupyter_black


In [2]:
experiment_dir = Path("./checkpoints/debug1")
with open(experiment_dir / "history/config.yaml", "r") as stream:
    cfg = yaml.safe_load(stream)

In [4]:
src_tokenizer = torch.load(experiment_dir / "src_tokenizer.pt")
tgt_tokenizer = torch.load(experiment_dir / "tgt_tokenizer.pt")
src_vocab = src_tokenizer.vocab
tgt_vocab = tgt_tokenizer.vocab

In [14]:
model = Seq2Seq(
    src_vocab_size=len(src_vocab),
    tgt_vocab_size=len(tgt_vocab),
    src_embedding_vector=None,
    tgt_embedding_vector=None,
    tgt_pad_index=tgt_vocab["<PAD>"],
    tgt_sos_index=tgt_vocab["<SOS>"],
    tgt_eos_index=tgt_vocab["<EOS>"],
    hidden_size=cfg["hidden_size"],
    bidirectional=cfg["bidirectional"],
    num_layers=cfg["num_layers"],
    src_embedding_size=cfg["src_embedding_size"],
    tgt_embedding_size=cfg["tgt_embedding_size"],
    dropout=cfg["dropout"],
)
model.to(device)

Seq2Seq(
  (src_embedding): Embedding(45000, 300)
  (tgt_embedding): Embedding(28000, 300)
  (encoder): Encoder(
    (embedding): Embedding(45000, 300)
    (layers): Sequential(
      (0): Embedding(45000, 300)
      (1): LSTM(300, 600, num_layers=2, batch_first=True, dropout=0.3, bidirectional=True)
    )
  )
  (decoder): Decoder(
    (embedding): Embedding(28000, 300)
    (lstm): LSTM(300, 600, num_layers=2, batch_first=True, dropout=0.3)
    (attention): Attention(
      (projection_layer): Linear(in_features=1200, out_features=600, bias=True)
    )
    (decoder_linear): Sequential(
      (0): Linear(in_features=1800, out_features=600, bias=True)
      (1): Tanh()
      (2): Linear(in_features=600, out_features=28000, bias=True)
    )
  )
)

In [15]:
model, _, _, epoch = load_checkpoint(model, experiment_dir / "model_best.pt")

In [17]:
%%time
with open(f"{data_root}/dev.src") as srcfile:
    sources = srcfile.readlines()

with open(f"{data_root}/dev.tgt") as tgtfile:
    references = tgtfile.readlines()

hypotheses = []
for source in tqdm(sources):
    hyp, _ = generate(model, source, src_tokenizer, tgt_tokenizer, cfg, method="beam")
    hypotheses.append(hyp[0])

metrics = compute_metrics(hypotheses, references)
metrics = {k: v * 100 for k, v in metrics.items()}
metrics

100%|██████████████████████████████████████████████████████████████████████████████████████████████| 10570/10570 [03:26<00:00, 51.24it/s]


CPU times: user 3min 46s, sys: 889 ms, total: 3min 47s
Wall time: 3min 49s


{'bleu1': 23.336224257946014,
 'bleu2': 9.72084030508995,
 'bleu3': 4.790161550045013,
 'bleu4': 2.58890837430954,
 'rougeL': 19.726860523223877}